In [34]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import numpy as np
import scipy.signal as sps
import soundfile as sf
import os
import math
import wave
from scipy.io import wavfile
from keras.models import load_model
from keras.models import Sequential
import xlsxwriter
import openpyxl
import random

from pesq import pesq, NoUtterancesError
from pystoi import stoi
import mir_eval

import matplotlib.pyplot as plt
from IPython.display import Audio
from torchaudio.utils import download_asset
import torch
import torchaudio
import torchaudio.functional as F

print(torch.__version__)
print(torchaudio.__version__)

%matplotlib inline
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

2.0.1+cpu
2.0.2+cpu
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [35]:
def create_spectrogram(audio_file, image_file):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

    y, sr = librosa.load(audio_file)
    ms = librosa.feature.melspectrogram(y=y, sr=sr, S=None, n_fft=1024, hop_length=80, win_length=320, window='hann', center=True, pad_mode='constant', power=2.0)
    #ms = librosa.feature.melspectrogram(y=y, sr=sr, S=None)
    log_ms = librosa.power_to_db(ms, ref=np.max)
    librosa.display.specshow(log_ms, sr=sr)

    fig.savefig(image_file)
    plt.close(fig)
    
def create_pngs_from_wavs(input_path, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    dir = os.listdir(input_path)

    for i, file in enumerate(dir):
        input_file = os.path.join(input_path, file)
        output_file = os.path.join(output_path, file.replace('.wav', '.png'))
        create_spectrogram(input_file, output_file)

In [36]:
def load_images_from_path(path, label, limit=1500):
    images = []
    labels = []

    file_list = os.listdir(path)
    if limit is not None:
        file_list = file_list[:limit]

    for file in file_list:
        images.append(img_to_array(load_img(os.path.join(path, file), target_size=(224, 224, 3))))
        labels.append(label)
        
    return images, labels

def show_images(images):
    fig, axes = plt.subplots(1, 8, figsize=(20, 20), subplot_kw={'xticks': [], 'yticks': []})

    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i] / 255)

In [37]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('models/my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 128)     36992     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       147584    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [38]:
path = "Spectrograms_wide_VA"

path_spec_test = path + "/ForTim/"

isExist = os.path.exists(path_spec_test)
if not isExist:
    # Create a new directory because it does not exist
    os.makedirs(path_spec_test)
    print("the %s directory is created!", path_spec_test)

noisy = ["-3dB", "-6dB", "-9dB", "0dB"]
clean = ["3dB", "6dB", "9dB", "clean"]
target_names = ["clean", "0dB", "-3dB", "-6dB", "-9dB", "3dB", "6dB", "9dB"]
user_target = ["0dB", "-3dB", "-6dB", "-9dB", "3dB", "6dB", "9dB"]

the %s directory is created! Spectrograms_wide_VA/ForTim/


In [39]:
ex_path = path_spec_test + "/combine.xlsx"

In [40]:
isExist3 = os.path.exists(ex_path)
if not isExist3:
   workbook1 = xlsxwriter.Workbook(ex_path)
   worksheet1 = workbook1.add_worksheet()
   worksheet1.write(0, 0, "Number")
   worksheet1.write(0, 1, "Name")
   worksheet1.write(0, 2, "Original SNR(dB)")
   worksheet1.write(0, 3, "Alpha Value")
   worksheet1.write(0, 4, "SNR(dB)")
   worksheet1.write(0, 5, "PESQ Evaluation")
   worksheet1.write(0, 6, "STOI Evaluation")
   worksheet1.write(0, 7, "SDR Evaluation")
   worksheet1.write(0, 8, "SI-SDR Evaluation")
   worksheet1.write(0, 9, "PESQ Highest")
   worksheet1.write(0, 10, "STOI Highest")
   print("The excel file 1 is created!")
   workbook1.close()

The excel file 1 is created!


In [41]:
def hann_window_a_signal(Windowed_data):
    Hann_window = sps.windows.hann(len(Windowed_data))
    Hann_Windowed_data = Hann_window * Windowed_data
    padded_signal = np.pad(Hann_Windowed_data, (0, 512), 'constant')
    Windowed_data_fft = np.fft.fft(padded_signal, 1024)
    return Windowed_data_fft

def perform_gan_noise_subtraction(input_audio_file, output_audio_file, alpha, Noise_gan):
    N_fft = 1024

    samplerate, data = wavfile.read(input_audio_file)
    Bit_Check = wave.open(input_audio_file, 'rb')
    bit_depth = Bit_Check.getsampwidth() * 8
    data = data / (2 ** (bit_depth - 1))
    Overlaps = math.floor(len(data) / 128)
    audio_ss = np.zeros(len(data))

    for No_of_overlaps in range(Overlaps - 5):
        Rectangular_windowed_signal = data[0 + 128 * No_of_overlaps:512 + 128 * No_of_overlaps]
        Estimated_noise_PSD = np.zeros(N_fft)
        GAN_noise_estimate = np.zeros(N_fft)
        FFT_of_windowed_signal = hann_window_a_signal(Rectangular_windowed_signal)

        Hann_window = sps.windows.hann(len(Rectangular_windowed_signal))
        PSD_window_scaling = np.sum(Hann_window ** 2)
        PSD_of_windowed_signal = (np.abs(FFT_of_windowed_signal) ** 2) / (samplerate * PSD_window_scaling)

        Tensor_PSD = tf.convert_to_tensor(PSD_of_windowed_signal.reshape(1, 1024), tf.float32)
        Generated_codebook = Noise_gan(Tensor_PSD)
        Generated_codebook = Generated_codebook.numpy()
        Generated_codebook_reshaped = np.abs((Generated_codebook.reshape(1024, 9)))

        Generated_codebook_inverse = np.linalg.pinv(Generated_codebook_reshaped, rcond=1e-15)
        Generated_coeffs = Generated_codebook_inverse * PSD_of_windowed_signal
        Generated_coeffs = np.transpose(Generated_coeffs)
        GAN_noise_codebook = (Generated_coeffs * Generated_codebook_reshaped)
        GAN_noise_codebook = GAN_noise_codebook.clip(min=0)

        for Freq_bin in range(0, N_fft):
            GAN_noise_estimate[Freq_bin] = np.sum(GAN_noise_codebook[Freq_bin, :])

        GAN_noise_estimate[512:1024] = np.flip(GAN_noise_estimate[0:512])
        scalar_factor_noise = sum([psd_value for psd_value in GAN_noise_estimate])
        GAN_noise_estimate_normalised = GAN_noise_estimate / scalar_factor_noise

        snr = PSD_of_windowed_signal / (100 * GAN_noise_estimate)
        Spectral_mask = 1 - np.minimum(1, np.maximum(0, alpha * snr))
        Clean_signal = FFT_of_windowed_signal * Spectral_mask
        Clean_frames = np.fft.ifft(Clean_signal)

        audio_ss[0 + 128 * No_of_overlaps:512 + 128 * No_of_overlaps] = audio_ss[
                                                                           0 + 128 * No_of_overlaps:512 + 128 * No_of_overlaps] + Clean_frames[0:512]

    # Save the processed audio
    sf.write(output_audio_file, audio_ss, samplerate, 'PCM_16')

https://pytorch.org/audio/0.13.1/tutorials/mvdr_tutorial.html

SI-SNR less undesired interference or distortion (HIGHER MEANS MORE SIMILAR TO THE REFERENCE SPEECH)

SDR means estimated speech contains more desired source signal and less undesired sources or noise (HIGHER FOR BETTER SPEECH QUALITY)

In [42]:
SAMPLE_RATE = 16000

def si_snr(estimate, reference, epsilon=1e-8):
    estimate = estimate - estimate.mean()
    reference = reference - reference.mean()
    reference_pow = reference.pow(2).mean(axis=1, keepdim=True)
    mix_pow = (estimate * reference).mean(axis=1, keepdim=True)
    scale = mix_pow / (reference_pow + epsilon)

    reference = scale * reference
    error = estimate - reference

    reference_pow = reference.pow(2)
    error_pow = error.pow(2)

    reference_pow = reference_pow.mean(axis=1)
    error_pow = error_pow.mean(axis=1)

    si_snr = 10 * torch.log10(reference_pow) - 10 * torch.log10(error_pow)
    return si_snr.item()

def evaluate(estimate, reference):
    try:
        si_snr_score = si_snr(estimate, reference)
        (
            sdr,
            _,
            _,
            _,
        ) = mir_eval.separation.bss_eval_sources(reference.numpy(), estimate.numpy(), False)
        pesq_mix = pesq(SAMPLE_RATE, estimate[0].numpy(), reference[0].numpy(), "wb")
        stoi_mix = stoi(reference[0].numpy(), estimate[0].numpy(), SAMPLE_RATE, extended=False)
        return pesq_mix, stoi_mix, sdr[0], si_snr_score
    except NoUtterancesError as e:
        return 0, 0  # Or any default values you prefer

In [43]:
# Load the noise GAN model
Noise_gan = tf.saved_model.load('../Spectogram/Full_Curriculum_4_generator')
# Load the clean audio folders
clean_audio_path = "../Spectogram/SNR_seg/"

In [44]:
# For one speech
#entries = ['speech_1000.wav']

In [32]:
# For spechifc amount of random speech
num_test = 3
# Create an empty array to store the random values
entries = []

# Generate 100 random values and append them to the list
for _ in range(num_test):
    random_value = random.randint(0, 824)
    entries.append(random_value)

# Sort the list in ascending order
entries.sort()

# Rename each item in the list to "speech_number.wav"
for i in range(len(entries)):
    entries[i] = "speech_" + str(entries[i]) + ".wav"


print(entries)

['speech_413.wav', 'speech_556.wav', 'speech_751.wav']


In [45]:
i = 0
for ent in entries:
    edit_ent = ent.replace(".wav", "")
    for target in user_target:
        os.makedirs(path_spec_test + "{}/".format(edit_ent + "(" + target + ")") + "spectrogram/", exist_ok=True)
        os.makedirs(path_spec_test + "{}/".format(edit_ent + "(" + target + ")") + "wav/", exist_ok=True)
        new_ent_img = path_spec_test + "{}/".format(edit_ent + "(" + target + ")") + "spectrogram/"
        new_ent_wav = path_spec_test + "{}/".format(edit_ent + "(" + target + ")") + "wav/"
        create_spectrogram(clean_audio_path + target + "/" + ent, new_ent_img + 'Default.png')
        create_spectrogram(clean_audio_path + "clean/" + ent, new_ent_img + 'clean.png')
        current_image = new_ent_img + 'Default.png'
        current_wav = clean_audio_path + target + "/" + ent
        audio_clean = False
        stop_value = 2
        start_value = 0
        clean_wav = clean_audio_path + "clean/" + ent
        counter = 0

        STOI_Highest = 0
        PESQ_Highest = 0
        
        while(audio_clean == False and start_value < stop_value):
            counter += 1
            # Preprocess the image
            image = img_to_array(load_img(current_image, target_size=(224, 224, 3)))
            image = image / 255.0
            image = np.expand_dims(image, axis=0)

            # Make predictions
            predictions = new_model.predict(image)
            # Find the index of the maximum prediction
            predicted_index = np.argmax(predictions)
            # Check if the predicted class matches the target
            predicted_class = target_names[predicted_index]

            start_value = round(start_value, 2)
            print("{}: Alpha value: {} Class: {}".format(edit_ent, "{:.2f}".format(start_value), predicted_class), end="\r")
            output_audio = new_ent_wav + 'Alpha{}.wav'.format("{:.2f}".format(start_value))
            perform_gan_noise_subtraction(current_wav, output_audio, start_value, Noise_gan)
            create_spectrogram(output_audio, new_ent_img + 'Alpha{}.png'.format("{:.2f}".format(start_value)))
            current_image = new_ent_img + 'Alpha{}.png'.format("{:.2f}".format(start_value))
            
            SAMPLE_CLEAN = clean_wav
            SAMPLE_NOISY = output_audio
            waveform_clean, sr = torchaudio.load(SAMPLE_CLEAN)
            waveform_noisy, sr2 = torchaudio.load(SAMPLE_NOISY)

            PESQ, STOI, SDR, SI_SNR_SCORE = evaluate(waveform_noisy[0:1], waveform_clean[0:1])

            if STOI > STOI_Highest:
                STOI_Highest = STOI
                STOI_saved = i 
            if PESQ > PESQ_Highest:
                PESQ_Highest = PESQ
                PESQ_saved = i

            workfile1 = openpyxl.load_workbook(ex_path)
            sheet1 = workfile1.active

            sheet1.cell(row=i+2, column=1).value = counter
            sheet1.cell(row=i+2, column=2).value = edit_ent
            sheet1.cell(row=i+2, column=3).value = target
            sheet1.cell(row=i+2, column=4).value = start_value
            sheet1.cell(row=i+2, column=5).value = predicted_class
            sheet1.cell(row=i+2, column=6).value = PESQ
            sheet1.cell(row=i+2, column=7).value = STOI
            sheet1.cell(row=i+2, column=8).value = SDR
            sheet1.cell(row=i+2, column=9).value = SI_SNR_SCORE
            workfile1.save(ex_path)
            i += 1

            if predicted_class in clean:
                audio_clean = True
            else: 
                start_value += 0.05

        workfile1 = openpyxl.load_workbook(ex_path)
        sheet1 = workfile1.active

        sheet1.cell(row=PESQ_saved+2, column=10).value = PESQ_Highest
        sheet1.cell(row=STOI_saved+2, column=11).value = STOI_Highest

        workfile1.save(ex_path)

        i += 1

        print("{}: Current SNR: {}, Alpha value: {}, Class: {}, PESQ: {}, STOI: {}".format(edit_ent, target, "{:.2f}".format(start_value), predicted_class, PESQ, STOI))

C:\Users\Edward\AppData\Local\Temp\ipykernel_13732\3105023457.py:51: ComplexWarning: Casting complex values to real discards the imaginary part
  audio_ss[0 + 128 * No_of_overlaps:512 + 128 * No_of_overlaps] = audio_ss[


speech_1000: Current SNR: 0dB, Alpha value: 2.00, Class: -3dB, PESQ: 1.1057612895965576, STOI: 0.48105509146517306
speech_1000: Current SNR: -3dB, Alpha value: 2.00, Class: -3dB, PESQ: 1.1057612895965576, STOI: 0.48105509146517306


KeyboardInterrupt: 

Spectral attenuate